<a href="https://colab.research.google.com/github/uhalpern/ICS-435-Final-Project/blob/main/Whisper_Finetuning_Korean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **ICS 435 Final - Fine-tuning Whisper From OpenAI**

Setup code for the model and datasets was adapted from this [blog post](https://huggingface.co/blog/fine-tune-whisper) on huggingface.

### **Preparing Environment**

In [1]:
# Run if you would like to work in Google Drive Directory

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
%cd /content/drive/MyDrive/Github/ICS-435-Final-Project

/content/drive/MyDrive/Github/ICS-435-Final-Project


In [37]:
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio evaluate

In [40]:
%cd /content/drive/MyDrive/Github/ICS-435-Final-Project
!pip install -r requirements.txt

/content/drive/MyDrive/Github/ICS-435-Final-Project
  Using cached https://pypi.nvidia.com/cudf-cu12/cudf_cu12-24.4.1-cp310-cp310-manylinux_2_28_x86_64.whl (473.3 MB)
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
Processing /colabtools/dist/google-colab-1.0.0.tar.gz (from -r requirements.txt (line 153))
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/colabtools/dist/google-colab-1.0.0.tar.gz'



## **Load Dataset Through Hugging Face**

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from datasets import load_dataset, DatasetDict

# Load the dataset
common_voice = load_dataset("mozilla-foundation/common_voice_17_0", "ko", split="train+validation+test", use_auth_token=True)

# Remove uneeded columns
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"])

# Randomly permute the dataset
common_voice = common_voice.shuffle(seed=42)

# Calculate the sizes for train, validation, and test sets
total_size = len(common_voice)
train_size = int(0.6 * total_size)
val_size = int(0.25 * total_size)
test_size = total_size - train_size - val_size

# Adjust sizes to include all examples
if train_size + val_size + test_size < total_size:
    test_size += total_size - (train_size + val_size + test_size)
elif train_size + val_size + test_size > total_size:
    # Adjust train size to avoid exceeding total size
    train_size -= (train_size + val_size + test_size) - total_size

# Create DatasetDict with train, validation, and test splits
splits = {
    "train": common_voice.select(range(train_size)),
    "validation": common_voice.select(range(train_size, train_size + val_size)),
    "test": common_voice.select(range(train_size + val_size, total_size))
}

common_voice = DatasetDict(splits)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2547: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for mozilla-foundation/common_voice_17_0 contains custom code which must be executed to correctly load the dataset. You 

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 376it [00:00, 68490.33it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 330it [00:00, 79993.08it/s]


...


Reading metadata...: 2057it [00:00, 109708.34it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 270it [00:00, 77795.02it/s]


Generating validated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1046it [00:00, 96616.13it/s]


Define feature extractor that pads or truncates audio inputs into 30 second segments. Then, the audio input is converted into log-Mel spectrogram form.

In [5]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Load Whisper tokenizer that maps token ids to corresponding text string.

In [6]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downsample audio to 16kHz as expected by the Whisper model

In [9]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

Function to prepare the dataset


1.   Resample audio to 16000 kHz
2.   Compute log-Mel spectrogram on input features in order to separate individual frequencies
3. Assign labels to transcriptions

In [10]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

Map the pre-processing function to the entire dataset

In [11]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/627 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/261 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/157 [00:00<?, ? examples/s]

## **Training and Evaluation**

In [13]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [14]:
# Define language and task for model
model.generation_config.language = "korean"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

Define data collator that will convert the input_features to batched PyTorch sensors. The labels will also be padded in order to match the padding given to the input_features. Padding tokens are represented by -100 so that it does not affect the loss.

In [15]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [16]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

Set up evaluation metrics

In [18]:
import evaluate

metric = evaluate.load("wer")

Define a function to return WER metric from model predictions to handle padded tokens. Decode the predicted and label ids to strings and compute the WER between predicted strings and labels.

In [19]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [20]:
# Create lists with three options for each hyperparameter
learning_rate_options = [1e-6, 1e-5, 1e-4]
warmup_steps_options = [10, 20, 30]
batch_gradient_tuples = [(32, 4), (64, 4), (128, 2)]
weight_decay_options = [0, 0.2, 0.3]
num_epochs_options = [5, 10, 15]

In [21]:
import random
from transformers import Seq2SeqTrainingArguments

random.seed(42)
training_args_list = []

# Perform random search with 5 passes
for i in range(5):
    # Randomly select hyperparameter values
    learning_rate = random.choice(learning_rate_options)
    warmup_steps = random.choice(warmup_steps_options)
    batch_size, gradient_accumulation_steps = random.choice(batch_gradient_tuples)
    weight_decay = random.choice(weight_decay_options)
    num_epochs = random.choice(num_epochs_options)

    total_examples = len(common_voice["train"])

    # Calculate steps per epoch based on batch size
    steps_per_epoch = total_examples // batch_size

    # Calculate total training steps
    total_steps = steps_per_epoch * num_epochs

    # Create model and evaluation checkpoints at half the total steps
    save_steps = total_steps // 2
    eval_steps = total_steps // 2

    training_args = Seq2SeqTrainingArguments(
        output_dir=f"./whisper-small-ko_pass_{i+1}",
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        warmup_steps=warmup_steps,
        max_steps=total_steps,
        gradient_checkpointing=True,
        fp16=False,
        evaluation_strategy="steps",
        per_device_eval_batch_size=4,
        predict_with_generate=True,
        generation_max_length=225,
        save_steps=save_steps,
        eval_steps=eval_steps,
        logging_steps=25,
        report_to=["tensorboard"],
        load_best_model_at_end=True,
        metric_for_best_model="wer",
        greater_is_better=False,
        push_to_hub=True,
        weight_decay=weight_decay
    )

    training_args_list.append(training_args)

    print(f"Pass {i+1}:")
    print(f"Learning Rate: {learning_rate}")
    print(f"Warmup Steps: {warmup_steps}")
    print(f"Batch Size: {batch_size}")
    print(f"Gradient Accumulation Steps: {gradient_accumulation_steps}")
    print(f"Weight Decay: {weight_decay}")
    print(f"Num Epochs: {num_epochs}")
    print("\n")

Pass 1:
Learning Rate: 0.0001
Warmup Steps: 10
Batch Size: 32
Gradient Accumulation Steps: 4
Weight Decay: 0.3
Num Epochs: 10


Pass 2:
Learning Rate: 1e-06
Warmup Steps: 10
Batch Size: 32
Gradient Accumulation Steps: 4
Weight Decay: 0.3
Num Epochs: 5


Pass 3:
Learning Rate: 0.0001
Warmup Steps: 30
Batch Size: 128
Gradient Accumulation Steps: 2
Weight Decay: 0
Num Epochs: 15


Pass 4:
Learning Rate: 1e-05
Warmup Steps: 10
Batch Size: 32
Gradient Accumulation Steps: 4
Weight Decay: 0
Num Epochs: 5


Pass 5:
Learning Rate: 1e-06
Warmup Steps: 30
Batch Size: 128
Gradient Accumulation Steps: 2
Weight Decay: 0
Num Epochs: 15




In [22]:
notebook_login()

In [24]:
from transformers import Seq2SeqTrainer

# Iterate over training arguments list and train the models
for i, training_args in enumerate(training_args_list):

    trainer = Seq2SeqTrainer(
        args=training_args,
        model=model,
        train_dataset=common_voice["train"],
        eval_dataset=common_voice["validation"],
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        tokenizer=processor.feature_extractor,
    )

    # Train the model
    print(f"Training model {i+1}...")
    trainer.train()

max_steps is given, it will override any value given in num_train_epochs


Training model 1...


OutOfMemoryError: CUDA out of memory. Tried to allocate 564.00 MiB. GPU 0 has a total capacity of 22.17 GiB of which 316.88 MiB is free. Process 13301 has 21.85 GiB memory in use. Of the allocated memory 21.02 GiB is allocated by PyTorch, and 610.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Evaluate The Base Model on The Held Out Test Set

In [27]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

max_steps is given, it will override any value given in num_train_epochs


In [28]:
trainer.evaluate()

{'eval_loss': 0.8617222905158997,
 'eval_wer': 39.06119027661358,
 'eval_runtime': 60.2073,
 'eval_samples_per_second': 2.608,
 'eval_steps_per_second': 0.664}

## Import Best Finetuned Model and Evaluate on Held Out Test Set

In [30]:
from transformers import WhisperForConditionalGeneration

# Replace "your-repo-name" with your actual repository name
model = WhisperForConditionalGeneration.from_pretrained("uhalpern/whisper-small-ko_pass_2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/2.25k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

In [31]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

max_steps is given, it will override any value given in num_train_epochs


In [32]:
trainer.evaluate()

{'eval_loss': 1.234740972518921,
 'eval_wer': 50.37720033528919,
 'eval_runtime': 62.4582,
 'eval_samples_per_second': 2.514,
 'eval_steps_per_second': 0.64}